In [1]:

import os, json

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory.chat_message_histories import ChatMessageHistory


from langchain_openai import ChatOpenAI



In [2]:

llm = ChatOpenAI(api_key="abc", base_url="http://localhost:8000/v1", model="Qwen2.5-7B-Instruct-AWQ",
                max_tokens=1024, temperature=0)






In [3]:

"""
对话聊天历史存储在redis中
https://api.python.langchain.com/en/latest/chat_message_histories/langchain_community.chat_message_histories.redis.RedisChatMessageHistory.html
https://blog.csdn.net/ppoojjj/article/details/141373205    # 实际使用案例
"""


from langchain_community.chat_message_histories.redis import RedisChatMessageHistory



# 创建历史会话记录
# https://python.langchain.com/docs/integrations/chat/vllm/

# #1.  测试单轮的LLM chain
chat_prompt_one_turn = ChatPromptTemplate.from_messages([("system", "你是一个有益的助手。"), 
                                                # MessagesPlaceholder(variable_name="history"), 
                                                ("user", "{input}")])

chain_one_turn = chat_prompt_one_turn | llm
print(llm.invoke("question数学运算：1+1="))
print(chain_one_turn.invoke({"input": "数学运算：1+1="}))

print("-------------------------------------------")
# # 2.  创建针对单个用户或者会话的多轮对话chain
chat_prompt = ChatPromptTemplate.from_messages(messages=[("system", "你是一个有益的助手。"), 
                                                         MessagesPlaceholder(variable_name="history"), 
                                                         ("human", "{question}")])

chain = chat_prompt | llm


chain_with_history =  RunnableWithMessageHistory(chain, lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
                                                input_messages_key="question",
                                                history_messages_key="history")




content='1+1=2' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 27, 'total_tokens': 33, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen2.5-7B-Instruct-AWQ', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7d806535-d053-4ee3-8b96-e9efe7482f97-0' usage_metadata={'input_tokens': 27, 'output_tokens': 6, 'total_tokens': 33, 'input_token_details': {}, 'output_token_details': {}}
content='1+1=2' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 26, 'total_tokens': 32, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen2.5-7B-Instruct-AWQ', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-70fa3abe-0361-4a36-8913-1359cda6aa66-0' usage_metadata={'input_tokens': 26, 'output_tokens': 6, 'total_tokens': 32, 'input_token_details': {}, 'output_

In [4]:

session_id = "111"

response_01 = chain_with_history.invoke({"question": "介绍你自己。"}, config={"configurable": {"session_id": session_id}})
print(response_01)

response_02 = chain_with_history.invoke({"question": "把你的回答整理成关键字的形式展示"}, config={"configurable": {"session_id": session_id}})
print(response_02)

"""
上述的输入可以采用循环的方式来模拟对话
"""






content='我是阿里云开发的一种人工智能助手，专注于生成高质量的文本内容。无论是解答问题、提供信息、进行对话，还是创造故事、提供建议，我都能以准确、有用且易于理解的方式回应您的需求。我的目标是成为您的知识伙伴，帮助您获取所需的信息，解决疑惑，提升工作效率。无论您需要学术知识、技术指南、生活建议还是娱乐内容，我都会尽力提供最合适的帮助。请随时向我提问，我在这里为您服务。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 160, 'total_tokens': 263, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen2.5-7B-Instruct-AWQ', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-108da541-97e2-4c8a-b238-f9860d5808ae-0' usage_metadata={'input_tokens': 160, 'output_tokens': 103, 'total_tokens': 263, 'input_token_details': {}, 'output_token_details': {}}
content='阿里云、人工智能助手、文本生成、问题解答、信息提供、对话、知识伙伴、获取信息、解决疑惑、提升效率、学术知识、技术指南、生活建议、娱乐内容、帮助、提问' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 280, 'total_tokens': 325, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qw

'\n上述的输入可以采用循环的方式来模拟对话\n'

In [5]:


"""
打印对话历史

"""
# 打印session_id=111的对话历史

session_history = RedisChatMessageHistory(session_id="111", url="redis://localhost:6379/0")
print(session_history.messages)


[HumanMessage(content='介绍你自己。', additional_kwargs={}, response_metadata={}), AIMessage(content='我是一个大型语言模型，由阿里云开发和维护。我被设计用来生成人类高质量的文本，回答各种问题、提供信息、进行对话等。我可以帮助解答问题、提供解释、创造故事、提供建议等。我的目标是为用户提供准确、有用且易于理解的回答。请随时向我提问，我会尽力提供帮助。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 22, 'total_tokens': 98, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen2.5-7B-Instruct-AWQ', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-defb5428-6a6c-4598-a874-a40b90cbc9df-0', usage_metadata={'input_tokens': 22, 'output_tokens': 76, 'total_tokens': 98, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='把你的回答整理成关键字的形式展示', additional_kwargs={}, response_metadata={}), AIMessage(content='大型语言模型、阿里云、文本生成、问题解答、信息提供、对话、帮助、准确、有用、易于理解、用户提问、提供帮助', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 115